In [1]:
from numpy.random import Generator, MT19937, PCG64
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind
from collections import Counter
import numpy as np
import random
import statistics

In [2]:
MT = Generator(MT19937()) # Mersenne Twister generator
PCG = Generator(PCG64()) # O’Neill’s permutation congruential generator

def game_of_life(m = 10, n = 10, show_plot = False, rng = MT):
    # initialize blank board
    base_population = np.zeros((m,n), dtype = int)
    
    # randomly select number of cells to be initialized with starting state alive or value of 1
    rand_num_ones = int(rng.random() * m * n)
    
    # generate random numbers for row and column indices
    rand_ms = rng.integers(low = 0, high = m, size = rand_num_ones)
    rand_ns = rng.integers(low = 0, high = n, size = rand_num_ones)
    
    # set random coordinates above equal to one in the base array of zeros
    alive = {}
    for i in range(rand_num_ones):
        base_population[rand_ms[i], rand_ns[i]] = 1  

    def cell_live_die_or_multiply(original_population):
        population = original_population.copy()
        iterations = 0
        SimulationComplete = False
        
        while True:
            iterations += 1
            if iterations == 1:
                second_last_population = np.zeros((population.shape[0], population.shape[1]))
                last_population = original_population.copy() # making copy to check if 2 states ago equals current state -- then in a repeating sequence
            elif iterations >= 2:
                second_last_population = last_population.copy()
                last_population = population.copy()
                
            for coordinate in alive:
                row = coordinate[0]
                col = coordinate[1]
                # set min and max row based on coordinates of cell and bounds of grid
                min_row = row - 1 if row - 1 >= 0 else row
                max_row = row + 1 if row + 1 < m else row
                min_col = col - 1 if col - 1 >= 0 else col
                max_col = col + 1 if col + 1 < n else col
                neighbor_count = sum([(population[min_row][col] == 1), (population[row][min_col] == 1), (population[min_row][min_col] == 1), \
                                        (population[max_row][min_col] == 1), (population[min_row][max_col] == 1), (population[max_row][max_col] == 1), \
                                        (population[max_row][col] == 1), (population[row][max_col] == 1)])
                
                # each cell with zero, one, or at least 4 neighbors dies
                if neighbor_count == 1 or neighbor_count == 0 or neighbor_count >= 4:
                    population[coordinate] = 0
            
            dead = np.where(population == 0)
            dead = zip(dead[0], dead[1])
            
            for coordinate in dead:
                dead_row = coordinate[0]
                dead_col = coordinate[1]
                # set min and max row based on coordinates of cell and bounds of grid
                min_row = dead_row - 1 if dead_row - 1 >= 0 else dead_row
                max_row = dead_row + 1 if dead_row + 1 < m else dead_row
                min_col = dead_col - 1 if dead_col - 1 >= 0 else dead_col
                max_col = dead_col + 1 if dead_col + 1 < n else dead_col
                neighbor_count = sum([(population[min_row][dead_col] == 1), (population[dead_row][min_col] == 1), (population[min_row][min_col] == 1), \
                                        (population[max_row][min_col] == 1), (population[min_row][max_col] == 1), (population[max_row][max_col] == 1), \
                                        (population[max_row][dead_col] == 1), (population[dead_row][max_col] == 1)])
                # each empty space with three neighbors becomes populated:
                if neighbor_count == 3:
                    population[coordinate] = 1
                    
            if show_plot == True: # display plot of current population (not an animation)
                plt.imshow(population, interpolation='nearest')
                plt.show()        
                
            # check conditions for simulation to end
            # all dead cells
            if (np.sum(population)) == 0: 
                SimulationComplete = True
            # the state from two iterations ago equals the current state
            elif iterations > 3 and (population == second_last_population).all() == True:
                SimulationComplete = True
            # if simulation complete, return iterations and break loop
            if SimulationComplete == True:
                return iterations
            # print(population)
        
    return "Simulation Ended",  cell_live_die_or_multiply(base_population), base_population

In [3]:
def eval_game(runs = 10000, rng_name = MT):
    trials = {}
    starting_states = []
    # getting estimate of average number of iterations before simulation is complete
    # using random board shapes between 3 and 100 rows and 3 and 100 columns
    for i in range(runs):
        game_round = game_of_life(np.random.randint(3, 100, 1)[0], np.random.randint(3, 100, 1)[0], rng_name)
        trials[i] = game_round[1]
        starting_states.append(''.join(map(str, game_round[2].flatten().tolist()))) # condense np array to list and join as string 
                                                                                  # to check for repeat starting states
    print(str(rng_name))      
    print('Max Non-Unique Starting State:', Counter(starting_states).most_common()[:1][0][1])
    print('Average Number of Iterations: ', sum(trials.values())/len(trials))
    print('Min Iterations: ', min(trials.values()))
    print('Max Iterations: ', max(trials.values()))
    print('Median Interations: ', statistics.median(sorted(trials.values())))
    return trials

In [4]:
# Mersenne Twister and PCG, 10000 runs
MT_samples = eval_game(rng_name = MT)
%timeit -n 1000 -r 7 game_of_life() # 7 runs, 1000 loops
print('\n')
PCG_samples = eval_game(rng_name = PCG)
%timeit -n 1000 -r 7 game_of_life(rng = PCG)

Generator(MT19937)
Max Non-Unique Starting State: 1
Average Number of Iterations:  10.4127
Min Iterations:  1
Max Iterations:  192
Median Interations:  7.0
2.19 ms ± 52.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Generator(PCG64)
Max Non-Unique Starting State: 2
Average Number of Iterations:  10.3483
Min Iterations:  1
Max Iterations:  164
Median Interations:  7.0
2.06 ms ± 278 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
# Eval difference in means
print('T-Test for whether means of the MT and PCG number of generations are equal')
t_statistic, p_value = ttest_ind(list(MT_samples.values()), list(PCG_samples.values())) # two-sample t-test
print("T-statistic:",  t_statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("There is a significant difference between the number of iterations of the PRN generators.")
else:
    print("There is no significant difference between the number of iterations of the PRN generators.")

T-Test for whether means of the MT and PCG number of generations are equal
T-statistic: 0.3713265609246933
p-value: 0.7103982508521827
There is no significant difference between the number of iterations of the PRN generators.
